In [71]:
import json
import requests

In [72]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [73]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io/legacy'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [74]:
def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

In [75]:
def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)

In [76]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [77]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

# EGFR inhibitors

Main query:



qg = {
  "nodes": {
    "n0": {
      "categories": [
        "biolink:ChemicalSubstance"
      ],
      "name": "Chemical Substance"
    },
    "n1": {
      "name": "EGFR",
      "ids": ["NCBIGene:1956"]
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicates": [
        "biolink:decreases_abundance_of",
        "biolink:decreases_activity_of",
        "biolink:decreases_expression_of",
        "biolink:decreases_synthesis_of",
        "biolink:increases_degradation_of",
        "biolink:disrupts",
        "biolink:negatively_regulates_entity_to_entity",
        "biolink:entity_negatively_regulates_entity"
      ]
    }
  }
}

EGFR_query = {'message':{'query_graph':qg}}

In [85]:
EGFR_result_id = submit_to_devars(EGFR_query)

https://arax.ncats.io/?source=ARS&id=73dde92f-59bd-4e84-bc32-750c1b79b33c


Probably the best query long term, as it removes a non-biolink predicate:

In [86]:
qg = {
  "nodes": {
    "n0": {
      "categories": [
        "biolink:ChemicalSubstance"
      ],
      "name": "Chemical Substance"
    },
    "n1": {
      "name": "EGFR",
      "ids": ["NCBIGene:1956"]
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicates": [
        "biolink:decreases_abundance_of",
        "biolink:decreases_activity_of",
        "biolink:decreases_expression_of",
        "biolink:decreases_synthesis_of",
        "biolink:increases_degradation_of",
        "biolink:disrupts",
        "biolink:negatively_regulates_entity_to_entity"
      ]
    }
  }
}

EGFR_query2 = {'message':{'query_graph':qg}}

In [87]:
EGFR_result_id2 = submit_to_devars(EGFR_query2)

https://arax.ncats.io/?source=ARS&id=d3205805-3251-4cc4-a420-646577b48066


Inverting edge and scaling up predicate to target molepro:

In [ ]:

qg = {
  "nodes": {
    "n0": {
      "categories": [
        "biolink:ChemicalSubstance"
      ],
      "name": "Chemical Substance"
    },
    "n1": {
      "name": "EGFR",
      "ids": ["NCBIGene:1956"]
    }
  },
  "edges": {
    "e0": {
      "subject": "n1",
      "object": "n0",
      "predicates": [
        "biolink:related_to"
      ]
    }
  }
}

EGFR_query3 = {'message':{'query_graph':qg}}

In [92]:
EGFR_result_id3 = submit_to_devars(EGFR_query3)

https://arax.ncats.io/?source=ARS&id=45105d5f-4b45-4b56-8c8a-d91494bc5f23


Using a Protein identifier rather than a gene identifier:

In [95]:
qg = {
  "nodes": {
    "n0": {
      "categories": [
        "biolink:ChemicalSubstance"
      ],
      "name": "Chemical Substance"
    },
    "n1": {
      "name": "EGFR",
      "ids": ["PR:000006933"]
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicates": [
        "biolink:negatively_regulates_entity_to_entity",
        "biolink:entity_negatively_regulates_entity"
      ]
    }
  }
}

EGFR_query_4 = {'message':{'query_graph':qg}}

In [96]:
EGFR_result_id4 = submit_to_devars(EGFR_query_4)

https://arax.ncats.io/?source=ARS&id=a548038a-a9e6-40f2-94ee-d6ba61a8d3b8


In [100]:

qg = {
"nodes": {
"n0": {
"categories": [
"biolink:ChemicalSubstance"
],
"name": "Chemical Substance"
},
"n1": {
"name": "EGFR",
"ids": ["NCBIGene:1956"]
}
},
"edges": {
"e0": {
"subject": "n0",
"object": "n1",
"predicates": [
"biolink:decreases_abundance_of",
"biolink:decreases_activity_of",
"biolink:decreases_expression_of",
"biolink:decreases_synthesis_of",
"biolink:increases_degradation_of",
"biolink:disrupts",
"biolink:negatively_regulates_entity_to_entity"
]
}
}
}
EGFR_query_redo = {'message':{'query_graph':qg}}

In [101]:
EGFR_result_redo = submit_to_devars(EGFR_query_redo)

https://arax.ncats.io/?source=ARS&id=36d9f32c-0427-463b-884c-c6de61752d92
